In [ ]:
#imports
import torch

In [ ]:
#download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
#read the text data into the variable 'text'
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
#see how long the dataset is
print(len(text))

In [ ]:
#see the first 1000 characters of text
print(text[:1000])

In [ ]:
#the set of characters in the dataset followed by the size of the set
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [ ]:
#create the character to integer mapping that will be necessary in tokenization
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] #encoder- takes a string then outputs a list of ints
decode = lambda l: ''.join([itos[i] for i in l]) #decoder- take the list of ints and output a string

In [ ]:
#test encode and decode
print(encode("i love transformers!"))
print(decode(encode("i love transformers!")))

In [ ]:
#encode the text and store it in a torch.Tensor
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000]) #the original dataset rerepresented as a sequence of integers

In [ ]:
#split the dataset into training and validation sets
n = int(0.9*len(data)) #90% train, 10% val
train_data = data[:n]
val_data = data[n:]

In [ ]:
#set the context window to train on
block_size = 8
train_data[:block_size+1]

In [ ]:
#create the context window that each batch will train on
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

In [ ]:
torch.manual_seed(1337)
batch_size = 4 #the number of independent sequences that will process in parallel
block_size = 8 #the maximum context length for predictions

def get_batch(split):
    #generate a small batch of data of inputs (x) and targets (y)
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

print("")

for b in range(batch_size): #batch dimension
    for t in range(block_size): #time dimension
        context = xb[b,:t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target is: {target}")

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        #each token reads off the logits for the next token using a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        #idc and targets are both (Batch,Time) tensor of integers
        logits = self.token_embedding_table(idx) #(Batch,Time,Channel)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
        
    def generate(self, idx, max_new_tokens):
        #idx is the (Batch, Time) array of indices in the current context
        for _ in range(max_new_tokens):
            #take the predictions
            logits, loss = self(idx)
            #focus only on the last time step
            logits = logits[:, -1, :] #becomes (Batch, Channel)
            #softmax values
            probs = F.softmax(logits, dim = 1) #(Batch, Channel)
            #sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(Batch, 1)
            #append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) #(Batch, Time+1)
        return idx
        
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens = 100)[0].tolist()))

In [ ]:
#create a PyTorch optimizer object
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [ ]:
batch_size = 32
for steps in range(1000):
    
    #sample a batch of data
    xb, yb = get_batch('train')
    
    #evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

In [ ]:
#see what kind of text we can generate
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens = 300)[0].tolist()))

# Mathematical trick in self-attention

In [ ]:
#consider the following example

torch.manual_seed(1337)
B, T, C = 4, 8, 2 #batch, time, channels
x = torch.randn(B,T,C)
x.shape

In [ ]:
#version 1
#we want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) #bow = bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(t, C)
        xbow[b,t] = torch.mean(xprev, 0) 
        #this method is slow and inefficient, we will instead use matrix multiplication 

In [ ]:
print(x[0])
print(xbow[0])

In [ ]:
#version 2
#perform weighted sums in a triangular form, token T will only 
#get information from other tokens preceding it
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim = True)
xbow2 = wei @ x #(B, T,T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow2)

In [ ]:
#version 3: using Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b #matrix multiplication
print('a =', a)
print('b =', b)
print('c =', c)

In [ ]:
#version 4: self-attention
torch.manual_seed(1337)
B,T,C = 4, 8, 32 #batch, time, channels
x = torch.randn(B,T,C)

#single head performs self attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) #B,T,16
q = query(x) #B,T,16

#make sure wei has variance 1, and not variance = batch_size
wei = q @ k.transpose(-2, -1)* head_size**-0.5 # B,T,16 @ B,16,T -> B,T,T


tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x #x is private to a v

out.shape


In [ ]:
 wei[0]
#No longer uniform lower triangular matrices, rather the weights are different on a
#batch by batch basis

In [ ]:
class LayerNorm1d: #used to be BatchNorm1d
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    #calculate the forward pass
    xmean = x.mean(1, keepdim=True) #calculate batch mean
    xvar = x.var(1, keepdim=True) #calculate batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) #normalize to variance = 1
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) #batch_size = 32 of 100-dim vectors
x = module(x)
x.shape